In [1]:
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import pandas as pd
from IPython.display import FileLink
import geopandas as gpd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import paramiko
from io import StringIO
from shapely.geometry import MultiPoint, MultiPolygon
import scipy
import scipy.cluster
from shapely.ops import unary_union
import calendar
from datetime import datetime

/home/rajat/anaconda3/envs/gisenv/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [3]:
def get_conn_postgres(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [4]:
def get_df_from_sql_postgres(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [10]:
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = "select warehouse_name, last_mile_hub, count(*) as orders from public.ops_main where date_trunc('month', created_date) >= date_trunc('month', now() - interval'1 month') and shipping_partner = 'Hyperlocal' and shipping_city = 'Bangalore' and warehouse_city = 'Bangalore' group by warehouse_name, last_mile_hub;"
# Establish a connection
conn = get_conn_postgres(SSH_required, key_path)

# Retrieve data into a DataFrame
df_ops_main = get_df_from_sql_postgres(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame
print(df_ops_main.head())

Tunnel Started
Connection Made
                                      warehouse_name last_mile_hub  orders
0  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          BLDR      97
1  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          CMRJ      11
2  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          ECTY      44
3  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          HBBL      45
4  15 Ground Floor, SY No  131 3, Hoskote, Anjane...          JPNR     100


In [8]:
df_warehouse = pd.read_csv('warehouses_w_lat_lng_blr.csv')
df_hubs = pd.read_csv('hubs_w_lat_lng_blr.csv')

In [9]:
print(df_warehouse.head())
print(df_hubs.head())

   warehouse_int_id  warehouse_id          warehouse_name  \
0              1467        Source                  Source   
1              1486           RK6                     RK6   
2                14  WE_Bangalore  WE Warehouse Bangalore   
3              1507            QA                      QA   
4              1543      Damensch                Damensch   

        warehouse_gstin  warehouse_phone warehouse_address warehouse_city  \
0       763872749872983  728197387298133            Source      Bangalore   
1  93289498239849234234       7893749879               RK6      Bangalore   
2                sodjgh       9455500191               NaN      Bangalore   
3       123232323245656             4545             DGDFG      Bangalore   
4                 NOGST       6363407643             Dummy      Bangalore   

  warehouse_state  warehouse_pincode     contact_name   contact_number wms_id  \
0       Karnataka             560037           Source  728197387298133      1   
1       

In [12]:
df_motherhub = df_hubs[(df_hubs['node_type'] == 'central_hub') & (df_hubs['contact_name'] == 'Biswojit Jena')]
print(df_motherhub)

   node_id  warehouse_id   node_name  location_id team_names  fuel_rate  \
6        1           182  DS BLR MTH            1     blr-lm          3   

   distance_limit sort_codes  \
6             100   BLR/MRTH   

                                             address   contact_name  \
6  WMXX+V8R, Marathahalli Village, Marathahalli, ...  Biswojit Jena   

   contact_phone    node_type locus_home_base_id  city_name actual_city_name  \
6     9066859089  central_hub          blr-lmh-4  Bangalore        Bangalore   

                   lat_long  
6  [12.9497375, 77.6982656]  


In [8]:
def total_distance(new_motherhub, orders, fixed_motherhub):
    total_dist = 0

    # Calculate combined distances for each order
    for _, row in orders.iterrows():
        client_location = (row['client_latitude'], row['client_longitude'])
        last_mile_location = (row['last_mile_latitude'], row['last_mile_longitude'])
        
        # Distances to fixed motherhub
        dist_client_to_fixed = great_circle(client_location, fixed_motherhub).km
        dist_fixed_to_last_mile = great_circle(fixed_motherhub, last_mile_location).km
        total_dist_fixed = dist_client_to_fixed + dist_fixed_to_last_mile
        
        # Distances to new motherhub
        dist_client_to_new = great_circle(client_location, new_motherhub).km
        dist_new_to_last_mile = great_circle(new_motherhub, last_mile_location).km
        total_dist_new = dist_client_to_new + dist_new_to_last_mile
        
        # Choose the smaller distance for this order
        total_dist += row['orders'] * min(total_dist_fixed, total_dist_new)
    
    return total_dist

In [ ]:
# Initial guess for new motherhub location (mean of existing locations)
initial_guess = (
    orders['client_latitude'].mean(),
    orders['client_longitude'].mean()
)

In [2]:
#cost should not go above a limit
#cost can be defined with rwith centre as centre

In [ ]:
# Optimization to find the best location for the new motherhub
result = minimize(total_distance, initial_guess, args=(orders, fixed_motherhub), method='Nelder-Mead')

In [ ]:
new_motherhub_location = result.x
print("Optimal location for new motherhub:", new_motherhub_location)